In [1]:
import numpy as np
from numpy import random as rd
import time
from solvers_listos import *
from funciones import *

In [2]:
def eliminar_duplicados(ruta):
    if type(ruta) != list:
        print(f'La ruta {ruta} no es una lista' )
        return []
    else:
        unique_nodes = []
        [unique_nodes.append(node) for node in ruta if node not in unique_nodes]
        return unique_nodes

def random_reverse(routes, nodos = [] ,iters = 1):
    rutas = routes.copy()
    for _ in range(iters):
        id = rd.choice(len(rutas))
        ruta = rutas[id]
        if len(ruta) > 4:
            i, j = rd.choice(range(1, len(ruta)-1), 2, replace=False)
            ruta[i], ruta[j] = ruta[j], ruta[i]
            # print(f'{id}: {ruta}')
        rutas[id] = eliminar_duplicados(ruta)
    return rutas

def random_swap(routes, nodos = [], iters=1):
    rutas = routes.copy()
    for _ in range(iters):
        id1,id2 = rd.choice(len(rutas), 2, replace = False)
        ruta1, ruta2 = rutas[id1], rutas[id2]
        # print(f'{id1}: {ruta1} | {id2}: {ruta2}')
        if len(ruta1) > 3 and len(ruta2)>3:
            bloque1 = rd.choice(range(1, len(ruta1)-2))  
            par1 = (ruta1[bloque1], ruta1[bloque1 + 1])
            bloque2 = rd.choice(range(1, len(ruta2)-2))
            par2 = (ruta2[bloque2], ruta2[bloque2 + 1])
            # print(f'R{id1}-Bloque1: {par1} x R{id2}-Bloque2: {par2}')
            ruta1[bloque1], ruta1[bloque1+1] = par2[0], par2[1] 
            ruta2[bloque2], ruta2[bloque2+1] = par1[0], par1[1]
            ruta1, ruta2 = eliminar_duplicados(ruta1), eliminar_duplicados(ruta2)
        rutas[id1] = ruta1
        rutas[id2] = ruta2
    return rutas

def random_move(routes, nodos = [], iters = 1):
    rutas = routes.copy()
    for _ in range(iters):
        id = rd.choice(len(rutas))
        ruta = rutas[id]
        if len(ruta) > 3:
            i, j = rd.choice(range(1, len(ruta)-1), 2, replace=False)
            node = ruta.pop(i)
            # print(f'Muevo el nodo {node} de la ruta {id} de la posición {i} a la posición {j}')
            ruta.insert(j, node)
        rutas[id] = eliminar_duplicados(ruta)
    return rutas

def random_insert(routes, nodos, iters = 1):
    rutas = routes.copy()
    for _ in range(iters):
        id = rd.choice(len(rutas))
        ruta = rutas[id]
        nodo = rd.choice(nodos)
        if ruta != [] and len(ruta)>2 and nodo != 'N_0':
            pos = rd.choice(range(1, len(ruta)))
            # print(f'Inserto el nodo {nodo} en la ruta {id} en la posición {pos}')
            ruta.insert(pos, nodo)
            ruta = eliminar_duplicados(ruta)
            rutas[id] = ruta
    return rutas

def random_remove(routes, nodos = [], iters = 1):
    rutas = routes.copy()
    for _ in range(iters):
        id = rd.choice(len(rutas))
        ruta = rutas[id]
        if ruta != [] and len(ruta) > 3:
            pos = rd.choice(range(1, len(ruta)-1))
            # print(f'Elimino el nodo {ruta[pos]} de la ruta {id} en la posición {pos}')
            ruta.pop(pos)
            rutas[id] = eliminar_duplicados(ruta)
    return rutas

def operacion_random(rutas, nodos):
    # ops = [random_insert, random_remove, random_move, random_swap, random_reverse]
    ops = [random_insert, random_remove, random_move, random_reverse]
    op = rd.choice(ops)
    # print(f'Operación: {op.__name__}')
    if len(rutas) != 0:
        resultado = op(rutas, nodos)
        if type(resultado)==list:
            r = [eliminar_duplicados(ruta) for ruta in resultado]
        elif type(resultado)==dict: 
            r = [eliminar_duplicados(ruta) for ruta in resultado.values()]
        return r
# nodos = [1,2,3,4,5,6,7,8,9,10]
# rutas = [[], [0,2,6,1,7,5], [0,4,1,7,5,3,9], [0,1,6,9,3,10,5]]
# operacion_random(rutas, nodos)


In [3]:
def realizacion_demanda_LS(G, demandas, ruido = 0.05):
    """
    Función que simula la demanda de los locales para un determinado periodo.
    """
    grafo = G.copy()
    insatisfecho = 0
    for nodo in grafo.nodes(data=True):
        if nodo[0] != 'N_0':
            dem = demandas[int(nodo[0][2:])]
            if dem <= grafo.nodes[nodo[0]]['Inv']:
                grafo.nodes[nodo[0]]['Inv'] -= dem

            else:
                grafo.nodes[nodo[0]]['Inv'] = 0
                insatisfecho += dem - grafo.nodes[nodo[0]]['Inv']
    # print(dems)
    # for nodo in grafo.nodes(data=True):
        # print(nodo[0],nodo[1]['Inv'])

    return grafo, insatisfecho

def adaptar_pron(prono, F):
    dict_pro = {}
    for f in range(F):
        dict_pro[f] = {nodo: prono[nodo]  for nodo in prono.keys()}
    return dict_pro

def reaccion_inventario_LS(graf, mu, sd, alfa = 0.025):
    """
    Función que verifica que locales deben ser visitados en base a su inventario actual. 
    En caso de que el inventario se encuentre bajo el umbral de tolerancia, se retorna True.
    """
    grafo = graf.copy()
    visitas = {nodo : False for nodo in G.nodes()}
    for nodo in grafo.nodes(data=True):
        id_nodo = int(nodo[0][2:])-1
        media = mu[id_nodo]
        desviacion = sd[id_nodo]
        s = media + norm.ppf((1 - alfa)/2)* desviacion  #Stock de seguridad
        # print(f'{nodo[1]["Inv"]}, s{int(nodo[0][2:])} = {s}, {nodo[1]["Inv"] <= s}')
        if nodo[1]["Inv"] <= s:
            visitas[nodo[0]] = True
            # print(f'Visitar {nodo[0]}')
    # print(visitas)
    return visitas


In [4]:
def costo_total(rutas, distancias, G, demandas):
    grafo = G.copy()
    costo_distancia = 0
    costo_SO = 0
    if type(rutas) == dict:
        w = 0
        for ruta in rutas.values():
            w+=1
            if ruta != [] and ruta != ['N_0'] and len(ruta) > 2:
                for i in range(len(ruta) - 1):
                    d = distancias[ruta[i]][ruta[i + 1]]
                    if d == np.inf:
                        print(f'{ruta} | {ruta[i]}, {ruta[i + 1]}: {d}')
                    costo_distancia += d/w
                # print(ruta)
                grafo, _ = ejecutar_ruta(grafo, ruta, distancias)
            else:
                costo_distancia += 0
            
            _, insatisfecho = realizacion_demanda_LS(grafo, demandas)
            costo_SO += insatisfecho*10000/w
            # print(ruta, costo_distancia, costo_SO)
    
    if type(rutas) == list:
        w = 0
        for ruta in rutas:
            w+=1
            if ruta != [] and ruta != ['N_0'] and len(ruta) > 2:
                for i in range(len(ruta) - 1):
                    d = distancias[ruta[i]][ruta[i + 1]]
                    if d == np.inf:
                        print(f'{ruta} | {ruta[i]}, {ruta[i + 1]}: {d}')
                    costo_distancia += d/w
                # print(ruta)
                grafo, _ = ejecutar_ruta(grafo, ruta, distancias)
            else:
                costo_distancia += 0
            
            _, insatisfecho = realizacion_demanda_LS(grafo, demandas)
            costo_SO += insatisfecho*10000/w
            # print(ruta, costo_distancia, costo_SO)
    return costo_SO
    # return costo_SO

In [5]:
def Local_Search(G, ruta_0, demandas, distancias, cap, F, n_restarts = 20, n_iters = 3):
    
    # time_limit = 10
    # t0 = time.time()
    best = ruta_0.copy()

    best_eval = costo_total(best, distancias, G, demandas) # acá no está tomando el costo de la ruta!!!
    print(f'Inicial LS: {best[0]}, costo SO = {best_eval}')
    nodos = G.nodes()

    # while time.time() - t0 < time_limit:
    for t in range(n_restarts):
        new = best.copy()
        for k in range(n_iters):
            new = operacion_random(new, nodos)
        new_eval = costo_total(new, distancias, G, demandas)

        # print(new, '|', best)
        # print(f'Candidata {t}: {new[0]}, costo SO = {new_eval}')
        # print(f'Costo nueva solución: {new_eval} | Costo mejor solución: {best_eval}')
        if new_eval < best_eval:
            best, best_eval = new, new_eval
            # print(f'MEJOR SOLUCION {t}, sol = {best[0]}, costo: {best_eval}')
        # else:
            # print(f'Mantengo {t}, sol = {best[0]}, costo: {best_eval}')
        # print(f'{t} | best: {best}, costo SO = {best_eval} ')    
    print(f'Final LS: {best}, costo SO = {best_eval}')
    # print(f'Costo final: {best_eval}, {best}')

    return best, best_eval


In [6]:
def ruteo_LS(H, distancias, pron_demandas, cap, F, mu, sd):
    grafo = H.copy()

    rutas = {}
    for t in range(F):
        '''
        Resolver el problema de ruteo para el periodo t
        Demanda = pronostico_demandas[t]
        '''
        demandas_t = pron_demandas[t]
        visitas_NN = reaccion_inventario_LS(grafo, mu, sd)
        # print([nodo for nodo in visitas_NN.keys() if visitas_NN[nodo]])
        if sum(visitas_NN.values()) == 0:
            # print("No hay locales que visitar")
            rutas[t] = []

        else:    
            ruta_R = nearest_neighbor(grafo, distancias, disponibilidad=visitas_NN)[-1] #devuelve la ruta a realizar
            rutas[t] = ruta_R
            # print(f"Ruta {t}: ", ruta_R )
            grafo, stock = ejecutar_ruta(grafo, ruta_R, distancias)   
        grafo, insatisfecho = realizacion_demanda_LS(grafo, demandas_t)
    
    print('\n')
    print('Ruteo LS')
    print(rutas)
    rutas, costo = Local_Search(grafo, rutas, demandas_t, distancias, cap, F)

    ruta_LS = eliminar_duplicados(rutas[0])

    return ruta_LS

In [23]:
def simular_ejecucion_P_LS_modificada(grafo_inicial, cap, dem_historico, T=1, F=1, tipo_demanda = 'n', d=30):
    # Inicializar variables     
    # ---------------------
    G0 = grafo_inicial.copy()
    distancias = calcular_matriz_dist(G0)
    ubicaciones = list(G0.nodes()) # Lista de ubicaciones
    inventarios = [G0.nodes(data=True)[i]['Inv'] for i in ubicaciones] # Lista de inventarios iniciales
    h = [G0.nodes(data=True)[i]['h'] for i in ubicaciones] # Lista de costos de inventario
     
    # ---------------------
    # Indicadores de desempeño        
    # ---------------------
    d_total = 0
    rutas = {t : None for t in range(T)} # Lista de rutas
    inventario_total = []
    perdidas = []
    c_rutas =[]
    demandas_efectivas = []
    costo_rutas = 0
    costo_SO = 0
    # ---------------------
    # Simulación
    # ---------------------
    
    for t in range(T):
        print('\n')
        mu_demanda = [np.mean(dem_historico[nodo]) for nodo in dem_historico.keys()]    
        sd_demanda = [np.std(dem_historico[nodo]) for nodo in dem_historico.keys()]
        pronostico = {int(nodo[2:]): pronostico_SEDA(
                                    dem_historico[nodo], T = F, pron = True, alpha=0.2, beta=0.1, theta=0.5)[0]
                                    for nodo in dem_historico.keys()}
        # print(pronostico)
        pronostico = adaptar_pron(pronostico, F)

        ruta_P = ruteo_LS(H = G0, distancias = distancias, pron_demandas = pronostico,
                           cap = cap, F = F, mu = mu_demanda, sd =sd_demanda)
        
        visitas_R = reaccion_inventario(G0, mu_demanda, sd_demanda)
        # print("Visitas reacción: ", visitas_reaccion)
        if sum(visitas_R.values()) == 0:
            ruta_R = []
        else:    
            ruta_R, largo_ruta_R = generar_ruta(G0, distancias, visitas_R, cap = cap) #devuelve la ruta a realizar
            
        print("REACTIVA", ruta_R)
        print("PROACTIVA: ", ruta_P)

        cr = calcular_largo_ruta(ruta_P, distancias)
        c_rutas.append(cr)
        costo_rutas += cr

        if ruta_P != [] and ruta_P != None and ruta_P != ['N_0']:
             ruta_P += ['N_0']
             G0, stock = ejecutar_ruta(G0, ruta_P, distancias)
        
        elif ruta_P == ['N_0'] or ruta_P == ['N_0','N_0']:
            ruta_P = []

        rutas[t] = ruta_P
        print(f"Ruta {t}: ", ruta_P)
        # visitas_proactiva = proactiva_inventario(G0, tolerancia = 0.2, dist = 'n', mu = 0, sigma = 0.1, M = 1000)

        G0, demanda, insatisfecho = realizacion_demanda_modificada(G0, dist = tipo_demanda, T=T, demandas_in=dem_historico, d=d, t=t)
        demandas_efectivas.append(demanda)
        costo_SO += insatisfecho*1
        d_total += sum(demanda.values())
        inventarios = [G0.nodes(data=True)[i]['Inv'] for i in ubicaciones if i != 'N_0']
        inventario_total.append(sum(inventarios))
        perdidas.append(insatisfecho)

        print(f'Tiempo: {t} | Ruta: {ruta_P} | costo_SO: {insatisfecho*1} | costo_r: {cr}')
        #Actualizo demandas
        for nodo in ubicaciones:
            if nodo != 'N_0':
                dem_historico[nodo].append(demanda[nodo]) 
                # HAY QUE VER CÓMO SE COMPORTA ESTO CON LA DEMANDA MODIFICADA

    # print('\n')
    # print("Inventario final: ")
    # for nodo in G0.nodes(data=True):
    #     print(nodo[0],nodo[1]['Inv'])
    print(f'F = {F}, Demanda perdida total: {sum(perdidas)} | Demanda perdida promedio: {sum(perdidas)/T}')        
    print(f'Costo ruta: {costo_rutas} | Costo SO: {costo_SO}')
    # graficar_rutas(rutas, G0)
    costos = (perdidas, c_rutas)
    return rutas, perdidas, inventario_total, costos, demandas_efectivas

In [25]:
np.random.seed(3)
G, ubis, cap_tpte, info_locales = crear_grafo_inicial(archivo= 'IRP1.xlsx' ,plot=False)
historia = simular_demanda_previa(G, dist = 'n', T=100) 
rutas, perdidas, inventarios, costos, dems = simular_ejecucion_P_LS_modificada(grafo_inicial = G, T = 30, dem_historico=historia, F = 5, cap = 871)






Ruteo LS
{0: ['N_0', 'N_2', 'N_0'], 1: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_6', 'N_5', 'N_0'], 2: ['N_0', 'N_2', 'N_1', 'N_7', 'N_8', 'N_0'], 3: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_6', 'N_5', 'N_0'], 4: ['N_0', 'N_2', 'N_1', 'N_0']}
Inicial LS: ['N_0', 'N_2', 'N_0'], costo SO = 6115913.668682318
Final LS: {0: ['N_0', 'N_2', 'N_0'], 1: ['N_0', 'N_8', 'N_4', 'N_5', 'N_6', 'N_0'], 2: ['N_0', 'N_2', 'N_1', 'N_7', 'N_8', 'N_0'], 3: ['N_0', 'N_1', 'N_9', 'N_10', 'N_3', 'N_6', 'N_5', 'N_0'], 4: ['N_0', 'N_1', 'N_2', 'N_0']}, costo SO = 6115913.668682318
REACTIVA ['N_0', 'N_2', 'N_0']
PROACTIVA:  ['N_0', 'N_2']
Ruta 0:  ['N_0', 'N_2', 'N_0']
Tiempo: 0 | Ruta: ['N_0', 'N_2', 'N_0'] | costo_SO: 240.7041866230989 | costo_r: 234.54637068179076




Ruteo LS
{0: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_7', 'N_6', 'N_0'], 1: ['N_0', 'N_2', 'N_1', 'N_8', 'N_5', 'N_0'], 2: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_7', 'N_6', 'N_0'], 3: ['N_0', 'N_2', 'N_1', 'N_5', 'N

In [26]:
def simular_ejecucion_paralela(grafo_inicial, cap, dem_historico, T=1, F=1, tipo_demanda = 'n', d=30):
    # Inicializar variables     
    # ---------------------
    G0 = grafo_inicial.copy()
    distancias = calcular_matriz_dist(G0)
    ubicaciones = list(G0.nodes()) # Lista de ubicaciones
    inventarios = [G0.nodes(data=True)[i]['Inv'] for i in ubicaciones] # Lista de inventarios iniciales
    h = [G0.nodes(data=True)[i]['h'] for i in ubicaciones] # Lista de costos de inventario
     
    # ---------------------
    # Indicadores de desempeño        
    # ---------------------
    d_total = 0
    rutas = {t : None for t in range(T)} # Lista de rutas
    inventario_total = []
    perdidas = []
    c_rutas =[]
    demandas_efectivas = []
    costo_rutas = 0
    costo_SO = 0
    # ---------------------
    # Simulación
    # ---------------------
    
    for t in range(T):
        print('\n')
        mu_demanda = [np.mean(dem_historico[nodo]) for nodo in dem_historico.keys()]    
        sd_demanda = [np.std(dem_historico[nodo]) for nodo in dem_historico.keys()]
        pronostico = {int(nodo[2:]): pronostico_SEDA(
                                    dem_historico[nodo], T = F, pron = True, alpha=0.2, beta=0.1, theta=0.5)[0]
                                    for nodo in dem_historico.keys()}
        # print(pronostico)
        pronostico = adaptar_pron(pronostico, F)

        ruta_P = ruteo_LS(H = G0, distancias = distancias, pron_demandas = pronostico,
                           cap = cap, F = F, mu = mu_demanda, sd =sd_demanda)
        
        visitas_R = reaccion_inventario(G0, mu_demanda, sd_demanda)
        # print("Visitas reacción: ", visitas_reaccion)
        if sum(visitas_R.values()) == 0:
            ruta_R = []
        else:    
            ruta_R, largo_ruta_R = generar_ruta(G0, distancias, visitas_R, cap = cap) #devuelve la ruta a realizar
            
        print("REACTIVA", ruta_R)
        print("PROACTIVA: ", ruta_P)

        cr = calcular_largo_ruta(ruta_P, distancias)
        c_rutas.append(cr)
        costo_rutas += cr

        if ruta_P != [] and ruta_P != None and ruta_P != ['N_0']:
             ruta_P += ['N_0']
             G0, stock = ejecutar_ruta(G0, ruta_P, distancias)
        
        elif ruta_P == ['N_0'] or ruta_P == ['N_0','N_0']:
            ruta_P = []

        rutas[t] = ruta_P
        print(f"Ruta {t}: ", ruta_P)
        # visitas_proactiva = proactiva_inventario(G0, tolerancia = 0.2, dist = 'n', mu = 0, sigma = 0.1, M = 1000)

        G0, demanda, insatisfecho = realizacion_demanda_modificada(G0, dist = tipo_demanda, T=T, demandas_in=dem_historico, d=d, t=t)
        demandas_efectivas.append(demanda)
        costo_SO += insatisfecho*1
        d_total += sum(demanda.values())
        inventarios = [G0.nodes(data=True)[i]['Inv'] for i in ubicaciones if i != 'N_0']
        inventario_total.append(sum(inventarios))
        perdidas.append(insatisfecho)

        print(f'Tiempo: {t} | Ruta: {ruta_P} | costo_SO: {insatisfecho*1} | costo_r: {cr}')
        #Actualizo demandas
        for nodo in ubicaciones:
            if nodo != 'N_0':
                dem_historico[nodo].append(demanda[nodo]) 
                # HAY QUE VER CÓMO SE COMPORTA ESTO CON LA DEMANDA MODIFICADA

    # print('\n')
    # print("Inventario final: ")
    # for nodo in G0.nodes(data=True):
    #     print(nodo[0],nodo[1]['Inv'])
    print(f'F = {F}, Demanda perdida total: {sum(perdidas)} | Demanda perdida promedio: {sum(perdidas)/T}')        
    print(f'Costo ruta: {costo_rutas} | Costo SO: {costo_SO}')
    # graficar_rutas(rutas, G0)
    costos = (perdidas, c_rutas)
    return rutas, perdidas, inventario_total, costos, demandas_efectivas

In [28]:
np.random.seed(3)
G, ubis, cap_tpte, info_locales = crear_grafo_inicial(archivo= 'IRP1.xlsx' ,plot=False)
historia = simular_demanda_previa(G, dist = 'n', T=100) 
rutas, perdidas, inventarios, costos, dems = simular_ejecucion_paralela(grafo_inicial = G, T = 30, dem_historico=historia, F = 5, cap = 871)






Ruteo LS
{0: ['N_0', 'N_2', 'N_0'], 1: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_6', 'N_5', 'N_0'], 2: ['N_0', 'N_2', 'N_1', 'N_7', 'N_8', 'N_0'], 3: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_6', 'N_5', 'N_0'], 4: ['N_0', 'N_2', 'N_1', 'N_0']}
Inicial LS: ['N_0', 'N_2', 'N_0'], costo SO = 6115913.668682318
Final LS: {0: ['N_0', 'N_2', 'N_0'], 1: ['N_0', 'N_8', 'N_4', 'N_5', 'N_6', 'N_0'], 2: ['N_0', 'N_2', 'N_1', 'N_7', 'N_8', 'N_0'], 3: ['N_0', 'N_1', 'N_9', 'N_10', 'N_3', 'N_6', 'N_5', 'N_0'], 4: ['N_0', 'N_1', 'N_2', 'N_0']}, costo SO = 6115913.668682318
REACTIVA ['N_0', 'N_2', 'N_0']
PROACTIVA:  ['N_0', 'N_2']
Ruta 0:  ['N_0', 'N_2', 'N_0']
Tiempo: 0 | Ruta: ['N_0', 'N_2', 'N_0'] | costo_SO: 240.7041866230989 | costo_r: 234.54637068179076




Ruteo LS
{0: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_7', 'N_6', 'N_0'], 1: ['N_0', 'N_2', 'N_1', 'N_8', 'N_5', 'N_0'], 2: ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_3', 'N_7', 'N_6', 'N_0'], 3: ['N_0', 'N_2', 'N_1', 'N_5', 'N